In [34]:
import os
import pickle
import numpy as np
# import cv2 as cv
# from PIL import Image, ImageEnhance
# import matplotlib .pyplot as plt
from itertools import permutations, combinations
# import scipy.stats as stats
import pandas as pd
# from pylatex import Document, Section, Subsection, Tabular, MultiColumn, MultiRow
import torch
# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# import numpy as np
# import matplotlib.cm as cm
from src.mr_utils import mrs

mr_num = len(mrs)
strength_max = 5
datasets = ['MNIST', 'Caltech256', 'VOC', 'COCO']
test_cases_num = {'MNIST': 10000, 'Caltech256': 3061, 'VOC':4952, 'COCO':40775}
models = {'MNIST': ['AlexNet'],
		'Caltech256': ['DenseNet121'],
		'VOC': ['MSRN'],
		'COCO': ['MLD']}
model_names = ['MNIST_AlexNet_9938', 'Caltech256_DenseNet121_6838', 'VOC_MSRN', 'COCO_MLD']


# RQ1 Validity

In [12]:
for i, dataset in enumerate(datasets):
	result_selfOracle = np.load(f'results/SelfOracle/{dataset}_validity.npy', allow_pickle=True).item()
	with open(f'results/SelfOracle/{dataset}_threshold.txt', 'r') as file:
		for line in file:
			if line.split(':')[0] == 'Threshold':
				th = float(line.split(':')[1])

	data = [[] for _ in range(strength_max)]
	for cmr in result_selfOracle.keys():
		if len(cmr) > strength_max:
			continue
		data[len(cmr)-1].append(len(np.where(np.array(result_selfOracle[cmr]) <= th)[0]) / 
									len(result_selfOracle[(0,)]) * 100)

	plt.figure(figsize=(3, 3))
	plt.boxplot(data, patch_artist=False)

	plt.xlabel(f'Composition strength ($k$)')
	plt.ylabel('Proportion of valid images')

	if i > 0:
		plt.ylim(96, 100.1)
		plt.yticks([96, 97, 98, 99, 100])  # 设定刻度

	plt.tight_layout()
	plt.savefig(f'figures/RQ1/{dataset}_validity.pdf', dpi=600)
	#plt.show()
	plt.close()
	# medians = [np.median(group) if group else np.nan for group in data]
	# for k, m in enumerate(medians, start=1):
	# 	print(data[k-1])
	# 	print(f"{dataset}: k={k}, median={m:.2f}%")

# RQ2

In [ ]:
def get_model_name(dataset, model):
	for i in range(len(model_names)):
		if dataset+'_'+model in model_names[i]:
			return model_names[i]

def get_validity(dataset):
	filename_validity = 'results/SelfOracle/' + dataset + '_validity.npy'
	filename_threshold = 'results/SelfOracle/' + dataset + '_threshold.txt'
	validity = np.load(filename_validity, allow_pickle=True).item()
	with open(filename_threshold) as f:
		lines = f.readlines()
		threshold = float(lines[1].split(':')[1].strip())
	# print(threshold)
	for mr in validity:
		for i in range(len(validity[mr])):
			if validity[mr][i] <= threshold:
				validity[mr][i] = True
			else:
				validity[mr][i] = False
	return validity


In [ ]:
# # Store errors

# def error_revealing(dataset, mr, pred_source, pred_followup, validity_followup):
# 	if dataset in  ['MNIST', 'Caltech256']:
# 		error = []
# 		for i in range(len(pred_source)):
# 			if validity_followup[mr][i] and (pred_followup[mr][i] != pred_source[i]):
# 				error.append(i)
# 	else:
# 		pred_f = pred_followup[mr]
# 		pred_f = pred_f.drop(columns=['img'])
# 		pred_f = pred_f.to_numpy()
# 		pred_f = {i: set(row[~pd.isna(row)]) for i, row in enumerate(pred_f)}
# 		error = []
# 		for i in range(len(pred_source)):
# 			if validity_followup[mr][i] and (pred_f[i] != pred_source[i]):
# 				error.append(i)
# 	return error

# for model_name in model_names:
# 	dataset = model_name.split('_')[0]
# 	strength = len(mrs)
# 	error = {}

# 	if dataset in ['MNIST', 'Caltech256']:
# 		pred_source = np.load(os.path.join('results', 'predictions', dataset, model_name+'_source.npy'))
# 	else:
# 		pred_source = pd.read_csv(os.path.join('results', 'predictions', dataset, model_name+'_source.csv'), low_memory=False)
# 		pred_source = pred_source.drop(columns=['img'])
# 		pred_source = pred_source.to_numpy()
# 		pred_source = {i: set(row[~pd.isna(row)]) for i, row in enumerate(pred_source)}

# 	pred_followup = np.load(os.path.join('results', 'predictions', dataset, model_name+'_followup.npy'),allow_pickle=True).item()
# 	#print(len(pred_followup))
# 	validity_followup = get_validity(dataset)
# 	for mr in pred_followup:
# 		error[mr] = error_revealing(dataset, mr, pred_source, pred_followup, validity_followup)
	
# 	with open(f'results/errors/error_{model_name}.pkl', 'wb') as f:
# 		pickle.dump(error, f)
# 	print(model_name, 'Done')

In [ ]:
# # save fault

# errors = {}
# for model_name in model_names:
# 	with open(f'results/errors/error_{model_name}.pkl', 'rb') as f:
# 		error = pickle.load(f)
# 		errors[model_name] = error

# for dataset in datasets:
# 	for index, model in enumerate(models[dataset]):
# 		model_name = get_model_name(dataset, model)

# 		pred_followup = np.load(os.path.join('results', 'predictions', dataset, model_name+'_followup.npy'),allow_pickle=True).item()
# 		if dataset in ['MNIST', 'Caltech256']:
# 			pred_source = np.load(os.path.join('results', 'predictions', dataset, model_name+'_source.npy'))
# 		else:
# 			pred_source = pd.read_csv(os.path.join('results', 'predictions', dataset, model_name+'_source.csv'), low_memory=False)
# 			pred_source = pred_source.drop(columns=['img'])
# 			pred_source = pred_source.to_numpy()
# 			pred_source = {i: tuple(sorted(set(row[~pd.isna(row)]))) for i, row in enumerate(pred_source)}
		
# 			for key in pred_followup:
# 				t = pred_followup[key]
# 				t = t.drop(columns=['img'])
# 				t = t.to_numpy()
# 				t = {i: tuple(sorted(set(row[~pd.isna(row)]))) for i, row in enumerate(t)}
# 				pred_followup[key] = t
# 		faults_cmr = {}
# 		faults_all = set()
# 		for i in range(strength_max):
# 			for cmr in permutations(range(len(mrs)), i + 1):
# 				failure = errors[model_name][cmr]
# 				faults = {}
# 				for f in failure:
# 					source_label = pred_source[f]
# 					followup_label = pred_followup[cmr][f]
# 					faults[f] = (source_label, followup_label)
# 				faults_cmr[cmr] = faults
# 				faults_all.update(faults.values())
# 		with open(f'results/errors/fault_{model_name}.pkl', 'wb') as f:
# 			pickle.dump(faults_cmr, f)
# 		print(dataset, model, len(faults_all))

In [24]:
errors = {}
for model_name in model_names:
    with open(f'results/errors/error_{model_name}.pkl', 'rb') as f:
        error = pickle.load(f)
        errors[model_name] = error

# RQ2

In [ ]:
# Compare the total failure ratio and the total number of faults

for dataset in datasets:
	for model in models[dataset]:
		print(dataset, model)
		model_name = get_model_name(dataset, model)
		validity_followup = get_validity(dataset)
		source_num = len(validity_followup[(0,)])
		with open(f'results/errors/fault_{model_name}.pkl', 'rb') as f:
			fault_all = pickle.load(f)
		cmr_num = 0
		failure_CMR_MAX, failure_CMR_MEAN = 0, 0
		fault_CMR_MAX, fault_CMR_MEAN = 0, 0

		fault_cmr, fault_max, fault_union = 0, 0, 0
		fault1, fault2, fault3, fault4, fault5 = 0, 0, 0, 0, 0

		for i in range(2, strength_max+1):
			for cmr in permutations(range(len(mrs)), i):
				cmr_num += 1
				failure_com, fault_com = [], []
				failure_com_union = set()
				fault_com_union = set()
				fault_com_max = set()
				for com in cmr:
					failure_com_union.update(errors[model_name][(com,)])
					failure_com.append(len(errors[model_name][(com,)]) / source_num * 100)
					fault_com.append(len(set(fault_all[(com,)].values())))
					fault_com_union.update(set(fault_all[(com,)].values()))
					if len(set(fault_all[(com,)].values())) > len(fault_com_max):
						fault_com_max = set(fault_all[(com,)].values())

				failure_CMR_MAX += len(errors[model_name][cmr]) / source_num * 100 - max(failure_com)
				failure_CMR_MEAN += len(errors[model_name][cmr]) / source_num * 100 - sum(failure_com)/i

				fault_cmr += len(set(fault_all[cmr].values()))
				fault_max += max(fault_com)
				fault_union += len(fault_com_union)

				fault1 += len(set(fault_all[cmr].values()) - fault_com_union)
				fault2 += len((set(fault_all[cmr].values()) & fault_com_union) - fault_com_max)
				fault3 += len(set(fault_all[cmr].values()) & fault_com_max)
				fault4 += len(fault_com_max - set(fault_all[cmr].values()))
				fault5 += len(fault_com_union - (set(fault_all[cmr].values()) | fault_com_max))

		dfr_max = failure_CMR_MAX / cmr_num
		dfr_mean = failure_CMR_MEAN / cmr_num
		dft_max = fault_CMR_MAX / cmr_num
		dft_mean = fault_CMR_MEAN / cmr_num
		fault_cmr, fault_max, fault_union = fault_cmr/cmr_num, fault_max/cmr_num, fault_union/cmr_num
		fault1, fault2, fault3, fault4, fault5 = fault1/cmr_num, fault2/cmr_num, fault3/cmr_num, fault4/cmr_num, fault5/cmr_num
		print(f" & {dfr_max:.2f}\% & {dfr_mean:.2f}\% & {dft_max:.1f} & {dft_mean:.1f} \\\\")
		print(f"{fault_cmr:.1f} {fault_max:.1f} {fault_union:.1f}")
		print(f"{fault1:.0f} {fault2:.0f} {fault3:.0f} {fault4:.0f} {fault5:.0f}")

	print('-------------------------------------------------------------------------------')


MNIST AlexNet
 & 0.98\% & 1.40\% & 0.0 & 0.0 \\
26.7 33.8 46.4
4 3 19 15 9
-------------------------------------------------------------------------------
Caltech256 DenseNet121
 & 10.27\% & 17.17\% & 0.0 & 0.0 \\
706.3 479.2 981.6
293 138 274 205 364
-------------------------------------------------------------------------------
VOC MSRN
 & 5.61\% & 10.38\% & 0.0 & 0.0 \\
399.9 321.5 512.1
101 66 232 89 124
-------------------------------------------------------------------------------
COCO MLD
 & 7.20\% & 24.07\% & 0.0 & 0.0 \\
18298.3 15941.0 34156.3
9827 2253 6219 9722 15963
-------------------------------------------------------------------------------


# RQ3

In [30]:
# draw box plot of Failure
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def compare_with_com_failure_4boxplot(dataset, model):
	model_name = get_model_name(dataset, model)
	validity_followup = get_validity(dataset)
	result_delta_max, result_delta_mean = [], []
	source_num = len(validity_followup[(0,)])
	for i in range(1, strength_max): 
		delta_max, delta_mean = [], []
		cmr_num = 0
		for cmr in permutations(range(len(mrs)), i + 1):
			cmr_num += 1
			cmr_failure_ratio = len(errors[model_name][cmr]) / source_num
			com_failure_ratio = []
			for mr in cmr:
				mr = (mr,)
				com_failure_ratio.append(len(errors[model_name][mr]) / source_num)
			delta_max.append(cmr_failure_ratio - max(com_failure_ratio))
			delta_mean.append(cmr_failure_ratio - sum(com_failure_ratio) / len(com_failure_ratio))
		result_delta_max.append(delta_max)
		result_delta_mean.append(delta_mean)
	return result_delta_max, result_delta_mean

def draw_boxplot_failure(dataset, model):
	delta_max, delta_mean = compare_with_com_failure_4boxplot(dataset, model)
	x_labels = [2, 3, 4, 5]
	group_labels = [r"$\mathit{DFR}_{\mathit{best}}$", r"$\mathit{DFR}_{\mathit{mean}}$"]

	records = []
	for i, x in enumerate(x_labels):
		for group, data in zip(group_labels, [delta_max[i], delta_mean[i]]):
			for value in data:
				records.append({'X': x, 'Group': group, 'Value': value})
	df = pd.DataFrame(records)
	# 绘图
	plt.figure(figsize=(4, 4))
	sns.boxplot(x='X', y='Value', hue='Group', data=df, palette='Set2', width=0.6, flierprops=dict(marker='o', markersize=3, linestyle='none') )
	#plt.title('Failure ' + dataset)
	plt.xlabel(f'$k$')
	plt.ylabel('Difference in failure ratio')
	plt.legend()
	plt.tight_layout()
	plt.savefig(f"figures/RQ3.1/failure_{dataset}_{model}_box_plot.pdf", dpi=300, bbox_inches='tight')
	#plt.show()
	plt.close()

	# print(f"\n===== Failure Ratio Difference: {dataset}, {model} =====")
	# for i, k in enumerate(x_labels):
	# 	for group_name, values in zip(group_labels, [delta_max[i], delta_mean[i]]):
	# 		if not values:
	# 			continue
	# 		series = pd.Series(values)
	# 		median = series.median()
	# 		q1 = series.quantile(0.25)
	# 		q3 = series.quantile(0.75)
	# 		min_val = series.min()
	# 		max_val = series.max()
	# 		neg_ratio = (series < 0).mean()
	# 		print(f"k = {k}, Group = {group_name}")
	# 		print(f"  Median = {median:.4f}, Q1 = {q1:.4f}, Q3 = {q3:.4f}")
	# 		print(f"  Min = {min_val:.4f}, Max = {max_val:.4f}, Negative ratio = {neg_ratio:.2%}")

for dataset in datasets:
	for model in models[dataset]:
		draw_boxplot_failure(dataset, model)

In [32]:
# draw box plot of Fault

def compare_with_com_fault_4boxplot(dataset, model):
	model_name = get_model_name(dataset, model)
	with open(f'results/errors/fault_{model_name}.pkl', 'rb') as f:
		fault_all = pickle.load(f)
	# for i in range(len(mrs)):
	# 	print(len(set(fault_all[(i,)])), end=' ')
	# print()
	result_delta_union, result_delta_max, result_delta_mean = [], [], []
	result_cmr_union_insert, result_cmr_union_diff = [], []
	for i in range(1, strength_max):
		delta_union, delta_max, delta_mean = [], [], []
		fault_cmr_union_insert, fault_cmr_minus_union, fault_union_minus_cmr = [], [], []
		for cmr in permutations(range(len(mrs)), i + 1):
			f_cmr = set(fault_all[cmr].values())
			com_fault = []
			com_fault_union = set()
			for mr in cmr:
				mr = (mr,)
				com_fault.append(len(set(fault_all[mr].values())))
				com_fault_union.update(fault_all[mr].values())

			delta_union.append(len(f_cmr) - len(com_fault_union))
			delta_max.append(len(f_cmr) - max(com_fault))
			delta_mean.append(len(f_cmr) - sum(com_fault) / len(com_fault))

			fault_cmr_union_insert.append(len(f_cmr & com_fault_union))
			fault_cmr_minus_union.append(len(f_cmr - com_fault_union))
		result_delta_union.append(delta_union)
		result_delta_max.append(delta_max)
		result_delta_mean.append(delta_mean)
		result_cmr_union_insert.append(fault_cmr_union_insert)
		result_cmr_union_diff.append(fault_cmr_minus_union)
	#return result_delta_union, result_delta_max, result_delta_mean, result_cmr_union_insert, result_cmr_union_diff
	return result_delta_max, result_delta_mean

def draw_boxplot_fault(dataset, model):
	delta_max, delta_mean = compare_with_com_fault_4boxplot(dataset, model)

	x_labels = [2, 3, 4, 5]
	group_labels = [r"$\mathit{DFT}_{\mathit{best}}$", r"$\mathit{DFT}_{\mathit{mean}}$"]

	records = []
	for i, x in enumerate(x_labels):
		for group, data in zip(group_labels, [delta_max[i], delta_mean[i]]):
			for value in data:
				records.append({'X': x, 'Group': group, 'Value': value})

	df = pd.DataFrame(records)

	plt.figure(figsize=(4, 4))
	sns.boxplot(x='X', y='Value', hue='Group', data=df, palette='Set2', width=0.6, flierprops=dict(marker='o', markersize=3, linestyle='none') )
	#plt.title('Failure ' + dataset)

	plt.xlabel(f'$k$')
	plt.ylabel('Difference in size of fault types')
	plt.legend()
	plt.tight_layout()
	plt.savefig(f"figures/RQ3.1/fault_{dataset}_{model}_box_plot.pdf", dpi=300, bbox_inches='tight')
	# plt.show()
	plt.close()

	# print(f"\n===== Fault Number Difference: {dataset}, {model} =====")
	# for i, k in enumerate(x_labels):
	# 	for group_name, values in zip(group_labels, [delta_max[i], delta_mean[i]]):
	# 		if not values:
	# 			continue
	# 		series = pd.Series(values)
	# 		median = series.median()
	# 		q1 = series.quantile(0.25)
	# 		q3 = series.quantile(0.75)
	# 		min_val = series.min()
	# 		max_val = series.max()
	# 		neg_ratio = (series < 0).mean()
	# 		print(f"k = {k}, Group = {group_name}")
	# 		print(f"  Median = {median:.4f}, Q1 = {q1:.4f}, Q3 = {q3:.4f}")
	# 		print(f"  Min = {min_val:.4f}, Max = {max_val:.4f}, Negative ratio = {neg_ratio:.2%}")

for dataset in datasets:
	for model in models[dataset]:
		draw_boxplot_fault(dataset, model)

# RQ3.2 Component

In [ ]:
# check delta Effectiveness w.r.t. Failure
from sklearn.decomposition import PCA
from scipy.stats import spearmanr

for dataset in datasets:
	filename = f'results/SelfOracle/{dataset}_validity.npy'
	validity_followup = get_validity(dataset) 
	if dataset == 'MNIST':
		extractor = 'lenet5'
	else:
		extractor = 'vgg16'
	save_path = os.path.join('results', 'features', extractor, f'{dataset}.pt')
	source__features = torch.load(save_path)
	followup__features_followup = []
	for i in range(len(mrs)):
		save_path = os.path.join('results', 'features', extractor, f'{dataset}_{i}.pt')
		followup__features_followup.append(torch.load(save_path))
	all_features = [source__features] + followup__features_followup
	all_features = [f.cpu().numpy() for f in all_features]
	all_features = np.concatenate(all_features, axis=0)
	pca = PCA(n_components=8)
	reduced_features = pca.fit_transform(all_features)
	source_length = len(source__features)
	source_features = reduced_features[:source_length]
	followup_features = reduced_features[source_length:]
	followup_separated = [followup_features[i * source_length:(i + 1) * source_length] for i in range(7)]
 
	for model in models[dataset]:
		print(dataset, model)
		model_name = get_model_name(dataset, model)
		for i in range(2, strength_max+1):
			deltadis, deltaeff = [], []
			for com in combinations(range(len(mrs)), i):
				select_com = com[0]
				for j in range(1, len(com)):
					if len(errors[model_name][(com[j],)]) / sum(validity_followup[(com[j],)]) > len(errors[model_name][(select_com,)]) / sum(validity_followup[(select_com,)]):
						select_com = com[j]
				delta_d = []
				delta_e = []
				for cmr in permutations(com):
					d, f, f_c, v_p_num = 0, 0, 0, 0
					for index in range(source_length):
						if validity_followup[cmr][index] and validity_followup[(select_com,)][index]:
							v_p_num += 1
							delta_com = []
							for c in cmr:
								delta_com.append(followup_separated[c][index] - source_features[index])
							combined = sum(delta_com)
							delta = followup_separated[select_com][index] - source_features[index]
							d += np.linalg.norm(combined) - np.linalg.norm(delta)

							if index in errors[model_name][(select_com,)]:
								f_c += 1
							if index in errors[model_name][cmr]:
								f += 1
					delta_d.append(d / v_p_num)
					delta_e.append((f - f_c) / v_p_num)
				deltadis.append(sum(delta_d) / len(delta_d))
				deltaeff.append(sum(delta_e) / len(delta_e))
			correlation, p_value = spearmanr(deltadis, deltaeff)
			# print(dataset, model, f'k={i}')
			# print('delta distance:', deltadis)
			# print('delta failure ratio:', deltaeff)
			# print(len(deltadis), correlation, p_value)
			plt.figure(figsize=(4, 3.5))
			plt.scatter(deltadis, deltaeff,  color="#002EA6", s=15)
			plt.xlabel(r"$\Delta_M$")
			plt.ylabel(r"$\mathit{DFR}_{\mathit{best}}$")

			plt.tight_layout()
			plt.savefig(f"figures/RQ3.2/failure_{dataset}_{model}_{i}_scatter_plot.pdf", dpi=300, bbox_inches='tight')
			#plt.show()
			plt.close()

In [ ]:
# check delta Effectiveness w.r.t. Fault
from sklearn.decomposition import PCA
from scipy.stats import spearmanr

for dataset in datasets:
	filename = f'results/SelfOracle/{dataset}_validity.npy'
	validity_followup = get_validity(dataset) 
	if dataset == 'MNIST':
		extractor = 'lenet5'
	else:
		extractor = 'vgg16'
	save_path = os.path.join('results', 'features', extractor, f'{dataset}.pt')
	source__features = torch.load(save_path)
	followup__features_followup = []
	for i in range(len(mrs)):
		save_path = os.path.join('results', 'features', extractor, f'{dataset}_{i}.pt')
		followup__features_followup.append(torch.load(save_path))
	all_features = [source__features] + followup__features_followup
	all_features = [f.cpu().numpy() for f in all_features]
	all_features = np.concatenate(all_features, axis=0)
	pca = PCA(n_components=8)
	reduced_features = pca.fit_transform(all_features)
	source_length = len(source__features)
	source_features = reduced_features[:source_length]
	followup_features = reduced_features[source_length:]
	followup_separated = [followup_features[i * source_length:(i + 1) * source_length] for i in range(7)]

	for model in models[dataset]:
		print(dataset, model)
		model_name = get_model_name(dataset, model)
		with open(f'results/errors/fault_{model_name}.pkl', 'rb') as f:
			fault_all = pickle.load(f)
		for i in range(2, strength_max+1):
			deltadis, deltaeff = [], []
			for com in combinations(range(len(mrs)), i):
				select_com = com[0]
				for j in range(1, len(com)):
					if (len(set(fault_all[(com[j],)].values())) > len(set(fault_all[(select_com,)].values()))):
						select_com = com[j]

				delta_d = []
				delta_e = []
				for cmr in permutations(com):
					d, f, f_c, v_p_num = 0, set(), set(), 0
					for index in range(source_length):
						if validity_followup[cmr][index] and validity_followup[(select_com,)][index]:
							v_p_num += 1

							delta_com = []
							for c in cmr:
								delta_com.append(followup_separated[c][index] - source_features[index])
							combined = sum(delta_com)
							delta = followup_separated[select_com][index] - source_features[index]
							d += np.linalg.norm(combined) - np.linalg.norm(delta)

							#print(cmr, d, index)
							if index in errors[model_name][(select_com,)]:
								f_c.add(fault_all[(select_com,)][index])
							if index in errors[model_name][cmr]:
								f.add(fault_all[cmr][index])
					delta_d.append(d / v_p_num)
					delta_e.append((len(f) - len(f_c)))
				#print(com, select_com, delta_d, delta_e)
				deltadis.append(sum(delta_d) / len(delta_d))
				deltaeff.append(sum(delta_e) / len(delta_e))
			correlation, p_value = spearmanr(deltadis, deltaeff)
			# print(dataset, model, f'k={i}')
			# print('delta distance:', deltadis)
			# print('delta fault number:', deltaeff)
			# print(len(deltadis), correlation, p_value)
			plt.figure(figsize=(4, 3.5))
			plt.scatter(deltadis, deltaeff,  color="#002EA6", s=15)
			plt.xlabel(r"$\Delta_M$")
			plt.ylabel(r"$\mathit{DFT}_{\mathit{best}}$")

			plt.tight_layout()
			plt.savefig(f"figures/RQ3.2/fault_{dataset}_{model}_{i}_scatter_plot.pdf", dpi=300, bbox_inches='tight')
			#plt.show()
			plt.close()

# RQ3.3 Sequence

In [26]:
def sequence_effect(dataset, model):
	model_name = get_model_name(dataset, model)
	print(model_name)
	validity_followup = get_validity(dataset)
	with open(f'results/errors/fault_{model_name}.pkl', 'rb') as f:
		fault_all = pickle.load(f)
	source_num = len(validity_followup[(0,)])
	for k in range(1, strength_max):
		k = k + 1
		dfr_max_mean, dfr_max_std, dfr_mean_mean, dfr_mean_std = [], [], [], []
		dft_max_mean, dft_max_std, dft_mean_mean, dft_mean_std = [], [], [], []
		for com in combinations(range(len(mrs)), k):
			dfr_max_com, dfr_mean_com, dft_max_com, dft_mean_com = [], [], [], []
			fr_max, fr_mean = 0, 0
			ft_max, ft_mean = 0, 0
			for mr in com:
				fr_mean += len(errors[model_name][(mr,)]) / source_num
				if len(errors[model_name][(mr,)]) / source_num > fr_max:
					fr_max = len(errors[model_name][(mr,)]) / source_num
				ft_mean += len(set(fault_all[(mr,)].values()))
				if len(set(fault_all[(mr,)].values())) > ft_max:
					ft_max = len(set(fault_all[(mr,)].values()))
			fr_mean, ft_mean = fr_mean / len(com), ft_mean / len(com)
			for per in permutations(com):
				error = errors[model_name][per]
				fr = len(error) / source_num
				ft = len(set(fault_all[per].values()))
				dfr_max_com.append(fr - fr_max)
				dfr_mean_com.append(fr - fr_mean)
				dft_max_com.append(ft - ft_max)
				dft_mean_com.append(ft - ft_mean)
			
			# dfr_max_cv.append(np.std(dfr_max_com)/np.mean(dfr_max_com))
			# dfr_mean_cv.append(np.std(dfr_mean_com)/np.mean(dfr_mean_com))
			# dft_max_cv.append(np.std(dft_max_com)/np.mean(dft_max_com))
			# dft_mean_cv.append(np.std(dft_mean_com)/np.mean(dft_mean_com))

			dfr_max_mean.append(np.mean(dfr_max_com))
			dfr_max_std.append(np.std(dfr_max_com))
			dfr_mean_mean.append(np.mean(dfr_mean_com))
			dfr_mean_std.append(np.std(dfr_mean_com))
			dft_max_mean.append(np.mean(dft_max_com))
			dft_max_std.append(np.std(dft_max_com))
			dft_mean_mean.append(np.mean(dft_mean_com))
			dft_mean_std.append(np.std(dft_mean_com))

		#print(len(dfr_max_cv), dfr_max_cv)
		dfr_max_mean, dfr_max_std, dfr_mean_mean, dfr_mean_std = np.mean(dfr_max_mean), np.mean(dfr_max_std), np.mean(dfr_mean_mean), np.mean(dfr_mean_std)
		dft_max_mean, dft_max_std, dft_mean_mean, dft_mean_std = np.mean(dft_max_mean), np.mean(dft_max_std), np.mean(dft_mean_mean), np.mean(dft_mean_std)
		print(f"& {k} & {dfr_max_mean*100:.2f}\\% & {dfr_max_std*100:.2f}\\% & {dfr_mean_mean*100:.2f}\\% & {dfr_mean_std*100:.2f}\\% & "
			f"{dft_max_mean:.2f} & {dft_max_std:.2f} & {dft_mean_mean:.2f} & {dft_mean_std:.2f} \\\\"
		)

def compare_sequence():
	for dataset in datasets:
		for _, model in enumerate(models[dataset]):
			sequence_effect(dataset, model)

In [27]:
compare_sequence()

MNIST_AlexNet_9938
& 2 & 0.12\% & 0.01\% & 0.32\% & 0.01\% & -1.21 & 0.45 & 6.74 & 0.45 \\
& 3 & 0.33\% & 0.04\% & 0.65\% & 0.04\% & -3.96 & 0.97 & 8.38 & 0.97 \\
& 4 & 0.67\% & 0.09\% & 1.07\% & 0.09\% & -6.18 & 1.58 & 8.62 & 1.58 \\
& 5 & 1.15\% & 0.20\% & 1.60\% & 0.20\% & -7.81 & 2.29 & 8.38 & 2.29 \\
Caltech256_DenseNet121_6838
& 2 & 2.35\% & 0.11\% & 5.82\% & 0.11\% & 62.33 & 3.81 & 152.19 & 3.81 \\
& 3 & 5.14\% & 0.19\% & 10.46\% & 0.19\% & 128.11 & 6.16 & 262.48 & 6.16 \\
& 4 & 8.22\% & 0.31\% & 14.70\% & 0.31\% & 190.84 & 10.30 & 351.24 & 10.30 \\
& 5 & 11.51\% & 0.41\% & 18.74\% & 0.41\% & 250.14 & 13.80 & 425.67 & 13.80 \\
VOC_MSRN
& 2 & 1.38\% & 0.09\% & 3.92\% & 0.09\% & 22.43 & 2.38 & 79.00 & 2.38 \\
& 3 & 2.90\% & 0.22\% & 6.71\% & 0.22\% & 43.71 & 3.30 & 124.20 & 3.30 \\
& 4 & 4.54\% & 0.35\% & 9.07\% & 0.35\% & 65.72 & 5.34 & 159.12 & 5.34 \\
& 5 & 6.26\% & 0.44\% & 11.22\% & 0.44\% & 86.39 & 7.06 & 187.39 & 7.06 \\
COCO_MLD
& 2 & 2.30\% & 0.10\% & 10.86\% & 0.10\% & 7